# mdr_text 프롬프팅 과정 (vLLM 버전)

In [ ]:
from typing import (
    Tuple,
    List,
    Dict,
    Any,
    Sequence,
    Union,
    Optional,
)

import sys
import time
import json
import re
from pathlib import Path
from enum import Enum
import shutil

import pandas as pd
import polars as pl
import polars.selectors as cs
import psutil

from tqdm import tqdm, trange
from pprint import pprint, pformat

# pydantic
from pydantic import BaseModel, Field, field_validator

# vLLM
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from vllm.sampling_params import StructuredOutputsParams

In [ ]:
import sys
from pathlib import Path

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# 이제 import
from src.loading import DataLoader
from src.utils import increment_path

loader = DataLoader(
    output_file= DATA_DIR / 'gold' / 'maude.parquet',
)

In [ ]:
adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
maude_lf = loader.load(adapter=adapter, **polars_kwargs)
maude_lf

In [ ]:
# Enum 정의
class PatientHarm(str, Enum):
    NO_HARM = "No Harm"
    MINOR_INJURY = "Minor Injury"
    SERIOUS_INJURY = "Serious Injury"
    DEATH = "Death"
    UNKNOWN = "Unknown"

class DefectType(str, Enum):
    FUNCTIONAL_FAILURE = "Functional Failure"
    MECHANICAL_STRUCTURAL = "Mechanical/Structural"
    ELECTRICAL_POWER = "Electrical/Power"
    SOFTWARE_INTERFACE = "Software/Interface"
    ALARM_ALERT = "Alarm/Alert"
    SENSOR_ACCURACY = "Sensor/Accuracy"
    COMMUNICATION_CONNECTIVITY = "Communication/Connectivity"
    LABELING_PACKAGING = "Labeling/Packaging"
    STERILITY_CONTAMINATION = "Sterility/Contamination"
    USER_HUMAN_FACTOR = "User/Human Factor"
    ENVIRONMENTAL_COMPATIBILITY = "Environmental/Compatibility"
    OTHER = "Other"
    UNKNOWN = "Unknown"

# BaseModel 정의
class IncidentDetails(BaseModel):
    patient_harm: PatientHarm = Field(description="Level of patient harm associated with the incident")
    problem_components: List[str] = Field(
        default_factory=list,
        description="List of problematic component keywords found in the text",
        min_length=0,
        max_length=5
    )
    incident_summary: str = Field(max_length=200, description="Concise summary of the incident")

class ManufacturerInspection(BaseModel):
    defect_confirmed: bool | None = Field(None, description="Whether the defect was confirmed")
    defect_type: DefectType | None = Field(None, description="Type of defect identified during inspection")
    inspection_actions: str | None = Field(None, max_length=200)

class MAUDEExtraction(BaseModel):
    incident_details: IncidentDetails
    manufacturer_inspection: ManufacturerInspection

SYSTEM_INSTRUCTION = SYSTEM_INSTRUCTION = """
You are an expert analyst of FDA MAUDE medical device adverse event reports.

Your mission is to extract high-quality structured variables from MDR text
by performing multi-dimensional decomposition and inference.

PRIMARY OBJECTIVE
- Produce exactly 6 derived variables with high semantic quality.
- Minimize the use of "Unknown" AND minimize defect_confirmed = null.
- If ANY symptom or contextual clue exists, you MUST infer the closest valid category.
- Express uncertainty via confidence_score, NOT via "Unknown" or null.

CORE PRINCIPLES
1. Symptom-driven inference over explicit statements.
2. Root-cause classification over surface wording.
3. UNKNOWN is a last resort, not a safe choice.
4. Regulatory / disclaimer boilerplate is background noise unless it contains incident facts.
5. Evidence-based outputs: every key value must be defensible from the text.
6. Operational usefulness: defect_confirmed should rarely be null.

UNKNOWN MINIMIZATION RULES
- patient_harm:
  * Use "Unknown" ONLY if the report gives absolutely no patient outcome information.
- defect_type:
  * Use "Unknown" ONLY if the MDR text provides virtually no describable symptom
    (e.g., only “device malfunctioned” with no further detail).
  * If symptoms exist but are ambiguous, choose the closest category and lower confidence_score.

DERIVED VARIABLES (MUST PRODUCE ALL 6)
A. patient_harm
   ["No Harm","Minor Injury","Serious Injury","Death","Unknown"]

B. defect_type
   One of:
   ["Functional Failure","Mechanical/Structural","Electrical/Power","Software/Interface",
    "Alarm/Alert","Sensor/Accuracy","Communication/Connectivity","Labeling/Packaging",
    "Sterility/Contamination","User/Human Factor","Environmental/Compatibility",
    "Other","Unknown"]

C. defect_confirmed (STRICT DECISION POLICY; null should be <5%)
Definition:
- "confirmed" means either:
  (1) Manufacturer/inspection explicitly confirmed the defect/cause, OR
  (2) The MDR provides symptom evidence that strongly supports the inferred defect_type
      (symptom-supported inference), even if manufacturer did not confirm.

You MUST assign one of: true / false / null using the rules below:

C1) Set defect_confirmed = true if ANY of the following is present:
- Explicit confirmation language:
  "confirmed", "found", "verified", "analysis determined", "root cause identified", "testing revealed"
- Corrective action that implies defect reality:
  "replaced", "removed", "repaired", "capped and replaced", "component exchanged", "device returned and found ..."
  AND the described symptoms are specific (not purely vague).
- Strong symptom specificity:
  Measurable/technical indicators (e.g., impedance high, threshold high, error code, fracture/leak, overheating,
  no power, calibration drift, lost connection) that map clearly to a defect_type.
Guideline:
- If defect_type is NOT "Unknown" and confidence_score >= 70, defect_confirmed should almost always be true.

C2) Set defect_confirmed = false if ANY of the following is present:
- Explicit non-confirmation / inconclusive language:
  "could not be confirmed", "no problem found", "no defect found", "unable to reproduce",
  "no anomalies detected", "no malfunction observed", "device met specifications"
- The report indicates suspicion only with weak support:
  phrases like "it was alleged", "it was suspected", "possibly", with no concrete symptom evidence,
  and confidence_score < 70.
- The report explicitly states investigation pending/under investigation AND provides no specific symptoms.

C3) Set defect_confirmed = null ONLY if ALL of the following are true (rare):
- No inspection/investigation status is mentioned (neither confirmed nor inconclusive nor pending), AND
- Symptom detail is too thin to support a strong inference (confidence_score < 40), AND
- defect_type must be "Unknown" or "Other" due to lack of describable symptom mapping.
In all other cases, you MUST choose true or false.

D. problem_components
   - Up to 5 concrete component keywords
   - Prefer physical or functional components (e.g., lead, battery, sensor, connector)
   - Avoid abstract or vague terms

E. incident_summary
   - ≤ 200 characters
   - Factual, concise
   - Include key symptom + action + patient outcome if available
   - No speculation or regulatory language

F. confidence_score
   Integer 0–100 indicating confidence in defect_type inference:
   - 90–100: explicit or highly specific symptom evidence
   - 70–89 : strong symptom evidence, no explicit confirmation
   - 40–69 : partial evidence, notable ambiguity
   - 0–39  : very thin evidence (if this low, reconsider whether Unknown is unavoidable)

EVIDENCE REQUIREMENT
- For each of A–E, provide one short evidence snippet (≤ 25 words).
- Prefer direct quotes from the MDR text.
- If inference is indirect, state:
  "No direct snippet; inferred from overall text" and lower confidence_score.

SELF-CHECK (MANDATORY)
Before finalizing the answer, perform the following internally:

[Step 1: Accuracy Check]
- Exactly 6 derived variables produced?
- UNKNOWN used only under strict rules?
- defect_type matches primary/root failure mode?
- defect_confirmed is null only under the strict C3 rule?
- Length limits respected?

[Step 2: Revision & Refinement]
- Fix any ambiguity, rule violations, or missing evidence.
- Improve conciseness and consistency.

Only after self-check, produce the final output.
"""


USER_PROMPT_TEMPLATE = """
Analyze this FDA MAUDE report and extract structured data:

[MDR TEXT]
{text}

[ORIGINAL PRODUCT PROBLEM]
{product_problem}

[OUTPUT REQUIREMENTS]
Return JSON with incident_details and manufacturer_inspection.

[FINAL REMINDER]
- UNKNOWN should appear in only exceptional cases.
- If symptoms exist, infer.
- Confidence belongs in confidence_score, not in UNKNOWN.
"""


In [ ]:
class BatchMAUDEExtractor:
    def __init__(self, 
                 model_path='Qwen/Qwen2.5-7B-Instruct',
                 tensor_parallel_size=1,
                 gpu_memory_utilization=0.9,
                 max_model_len=8192,
                 batch_size=32,
                 max_retries=2):
        """
        vLLM 최적화 배치 추출기
        
        Args:
            model_path: 모델 경로 (HuggingFace 또는 로컬)
            tensor_parallel_size: 사용할 GPU 수
            gpu_memory_utilization: GPU 메모리 사용률
            max_model_len: 최대 시퀀스 길이
            batch_size: 배치 크기
            max_retries: 재시도 횟수
        """
        self.batch_size = batch_size
        self.max_retries = max_retries
        self.model_path = model_path
        
        print(f"Loading vLLM model: {model_path}...")
        
        # vLLM 모델 초기화
        self.llm = LLM(
            model=model_path,
            tensor_parallel_size=tensor_parallel_size,
            gpu_memory_utilization=gpu_memory_utilization,
            max_model_len=max_model_len,
            trust_remote_code=True,
            enforce_eager=False,  # CUDA graph 사용
        )
        
        # Tokenizer 로드 (chat template 적용용)
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            trust_remote_code=True
        )
        
        print("Model loaded successfully!")
        
        self.json_schema = MAUDEExtraction.model_json_schema()
        # Sampling parameters with guided JSON
        self.sampling_params = SamplingParams(
            temperature=0.1,
            max_tokens=512,
            top_p=0.95,
            # Guided JSON decoding - 스키마에 맞는 JSON만 생성
            structured_outputs=StructuredOutputsParams(
                json=self.json_schema,
            )
        )

    def _create_prompts(self, rows: List[pd.Series]) -> List[str]:
        """Chat template을 적용한 프롬프트 생성"""
        prompts = []
        
        for row in rows:
            text = row['mdr_text']
            product_problem = row['product_problems']
            
            user_content = USER_PROMPT_TEMPLATE.format(
                text=text,
                product_problem=product_problem
            )
            
            # Chat template 적용
            messages = [
                {"role": "system", "content": SYSTEM_INSTRUCTION},
                {"role": "user", "content": user_content}
            ]
            
            # Tokenizer의 chat template 사용
            formatted_prompt = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            
            prompts.append(formatted_prompt)
        
        return prompts

    def _parse_and_validate(self, response_text: str) -> dict:
        """응답 파싱 및 검증"""
        # Guided JSON이므로 이미 JSON 형태
        data = json.loads(response_text)
        validated = MAUDEExtraction(**data)
        return validated.model_dump()

    def extract_batch(self, rows: List[pd.Series]) -> List[dict]:
        """
        vLLM 배치 추론
        - 순수 vLLM 배치 처리만 사용
        - Guided JSON으로 파싱 에러 최소화
        """
        # 프롬프트 생성
        prompts = self._create_prompts(rows)
        
        # vLLM 배치 추론 (여기서 자동으로 최적화됨)
        outputs = self.llm.generate(prompts, self.sampling_params, use_tqdm=False)
        
        # 결과 파싱
        results = []
        for i, output in enumerate(outputs):
            try:
                response_text = output.outputs[0].text
                validated_data = self._parse_and_validate(response_text)
                
                result = {
                    **validated_data,
                    '_row_id': rows[i].name,
                    '_success': True,
                    '_input_tokens': len(output.prompt_token_ids),      # 추가
                    '_output_tokens': len(output.outputs[0].token_ids),  # 기존
                    '_total_tokens': len(output.prompt_token_ids) + len(output.outputs[0].token_ids)  # 추가
                }
                results.append(result)
                
            except Exception as e:
                results.append({
                    '_row_id': rows[i].name,
                    '_success': False,
                    '_error': str(e)[:200],
                    '_raw_response': output.outputs[0].text[:200]
                })
        
        return results

    def process_with_retry(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        재시도 로직 포함 배치 처리
        - 실패한 항목들을 모아서 재배치 처리
        """
        all_results = []
        pending_rows = [(i, row) for i, row in df.iterrows()]
        attempt = 1
        
        while pending_rows and attempt <= self.max_retries:
            # print(f"\nAttempt {attempt}: Processing {len(pending_rows)} rows")
            
            # 배치 단위로 처리
            num_batches = (len(pending_rows) - 1) // self.batch_size + 1
            batch_results = []
            
            for batch_idx in range(num_batches):
                start_idx = batch_idx * self.batch_size
                end_idx = min((batch_idx + 1) * self.batch_size, len(pending_rows))
                
                batch_items = pending_rows[start_idx:end_idx]
                batch_rows = [row for _, row in batch_items]
                
                # vLLM 배치 추론
                results = self.extract_batch(batch_rows)
                
                for j, result in enumerate(results):
                    result['_attempts'] = attempt
                    batch_results.append((batch_items[j][0], result))
            
            # 성공/실패 분리
            success_results = [(idx, r) for idx, r in batch_results if r['_success']]
            failed_items = [(idx, df.loc[idx]) for idx, r in batch_results if not r['_success']]
            
            # 성공한 결과 저장
            all_results.extend([r for _, r in success_results])
            
            # 통계
            success_count = len(success_results)
            failed_count = len(failed_items)
            print(f"Success: {success_count}, Failed: {failed_count}")
            
            # 다음 시도를 위해 실패한 항목 설정
            pending_rows = failed_items
            attempt += 1
        
        # 최종 실패 항목 처리
        for idx, row in pending_rows:
            all_results.append({
                '_row_id': idx,
                '_success': False,
                '_error': 'Max retries exceeded',
                '_attempts': self.max_retries
            })
        
        # row_id 순서대로 정렬
        all_results.sort(key=lambda x: x['_row_id'])
        
        return pd.json_normalize(all_results)

    def process_batch(self, 
                     df: pd.DataFrame, 
                     checkpoint_dir: Union[str|Path], 
                     checkpoint_interval: int = 1000,
                     checkpoint_prefix: str = 'checkpoint',
        ) -> pd.DataFrame:
        """
        전체 데이터프레임 처리 with 체크포인트
        """
        print(f"="*60)
        print(f"vLLM Batch Processing")
        print(f"="*60)
        print(f"Total records: {len(df):,}")
        print(f"Batch size: {self.batch_size}")
        print(f"Max retries: {self.max_retries}")
        print(f"Checkpoint every: {checkpoint_interval} records\n")
        
        overall_start = time.time()
        all_results = []
        
        # 체크포인트 단위로 처리
        try:
            num_chunks = (len(df) - 1) // checkpoint_interval + 1
            
            for chunk_idx in tqdm(range(num_chunks), desc="Processing chunks"):
                start_idx = chunk_idx * checkpoint_interval
                end_idx = min((chunk_idx + 1) * checkpoint_interval, len(df))
                chunk_df = df.iloc[start_idx:end_idx]
                
                # print(f"\n{'='*60}")
                # print(f"Chunk {chunk_idx + 1}/{num_chunks}: Rows {start_idx:,}-{end_idx-1:,}")
                # print(f"{'='*60}")
                
                chunk_start = time.time()
                
                # 재시도 포함 처리
                chunk_result_df = self.process_with_retry(chunk_df)
                all_results.append(chunk_result_df)
                
                # 청크 통계
                elapsed = time.time() - chunk_start
                success = chunk_result_df['_success'].sum()
                throughput = len(chunk_df) / elapsed
                
                # print(f"\nChunk completed:")
                # print(f"  Success: {success}/{len(chunk_df)} ({100*success/len(chunk_df):.1f}%)")
                # print(f"  Time: {elapsed:.1f}s")
                # print(f"  Throughput: {throughput:.2f} samples/s")
                
                # 체크포인트 저장
                checkpoint_file = f'{checkpoint_prefix}_chunk{chunk_idx+1}.csv'
                checkpoint_path = Path(checkpoint_dir) / checkpoint_file
                chunk_result_df.to_csv(checkpoint_path, index=False)
                # print(f"  Checkpoint: {checkpoint_file}")
            
            # 최종 결과 합치기
            final_df = pd.concat(all_results, ignore_index=True)
            
            # 최종 통계
            total_time = time.time() - overall_start
            total_success = final_df['_success'].sum()
            
            print(f"\n{'='*60}")
            print(f"FINAL RESULTS")
            print(f"{'='*60}")
            print(f"Total processed: {len(final_df):,}")
            print(f"Success: {total_success:,} ({100*total_success/len(final_df):.1f}%)")
            print(f"Failed: {len(final_df)-total_success:,}")
            print(f"Total time: {total_time/60:.1f} min")
            print(f"Throughput: {len(final_df)/total_time:.2f} samples/s")
            print(f"Total tokens: {final_df['_total_tokens'].sum():,}")
            print(f"Avg input: {final_df['_input_tokens'].mean():.1f}")
            print(f"Avg output: {final_df['_output_tokens'].mean():.1f}")
            print(f"{'='*60}")
            
            return final_df
        
        finally:
            # 5. 임시 파일 정리
            if checkpoint_dir.exists():
                shutil.rmtree(checkpoint_dir)

In [ ]:
sampled_df = maude_lf.select(
    pl.all().sample(
        n=1000,
        with_replacement=False,
        shuffle=True, # Shuffle the order of sampled rows
        seed=4242
    )
).collect().to_pandas()

sampled_df.head()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
start_time = time.time()

extractor = BatchMAUDEExtractor(
    model_path='Qwen/Qwen3-8B',  # 또는 로컬 경로
    tensor_parallel_size=1,  # GPU 개수
    batch_size=32,  # 배치 크기
    max_retries=2
)

checkpoint_dir = DATA_DIR / 'temp'
# 처리
result_df = extractor.process_batch(sampled_df, checkpoint_interval=100, checkpoint_dir=checkpoint_dir)

elapsed = time.time() - start_time
print(f"Elapsed time: {elapsed:.2f} seconds")

In [ ]:
print(f"\n{'='*60}")
print(f"FINAL RESULTS")
print(f"{'='*60}")
print(f"Total processed: {len(result_df):,}")
print(f"Total tokens: {result_df['_total_tokens'].sum():,}")
print(f"Avg input: {result_df['_input_tokens'].mean():.1f}")
print(f"Avg output: {result_df['_output_tokens'].mean():.1f}")
print(f"{'='*60}")

In [ ]:
# 필요한 열만 선택 후 열 이름 변경
result_df2 = result_df[[
    'incident_details.patient_harm',
    'incident_details.problem_components',
    'incident_details.incident_summary',
    'manufacturer_inspection.defect_confirmed',
    'manufacturer_inspection.defect_type',
    'manufacturer_inspection.inspection_actions'
    ]]

result_df2 = result_df2.rename(columns={
    'incident_details.patient_harm': 'patient_harm',
    'incident_details.problem_components': 'problem_components',
    'incident_details.incident_summary': 'incident_summary',
    'manufacturer_inspection.defect_confirmed': 'defect_confirmed',
    'manufacturer_inspection.defect_type': 'defect_type',
    'manufacturer_inspection.inspection_actions': 'inspection_actions'
})

result_df2.head(100)

In [ ]:
result_df2['defect_type'].value_counts()

In [ ]:
result_df2['defect_confirmed'].value_counts()

In [ ]:
df_concat = pd.concat([sampled_df, result_df2], axis=1)
df_concat[['mdr_text', 'patient_harm', 'defect_type']]

save_path = DATA_DIR / 'adhoc' / 'maude_extracted_sample.csv'
save_path = increment_path(save_path)
df_concat.to_csv(save_path, index=False)

## vLLM 버전의 주요 개선사항

### 1. 성능 향상
- **배치 추론**: vLLM의 네이티브 배치 처리로 처리 속도 대폭 향상
- **PagedAttention**: 메모리 효율적인 attention 메커니즘
- **Continuous Batching**: 동적 배치 스케줄링으로 처리량 최적화

### 2. GPU 활용 최적화
- Tensor Parallelism 지원 (다중 GPU)
- 높은 GPU 메모리 활용률 (기본 0.9)
- 효율적인 KV 캐시 관리

### 3. 처리 속도 비교 (예상)
- **Ollama 버전**: ~1-2 samples/s (CPU 또는 단일 GPU)
- **vLLM 버전**: ~10-50 samples/s (GPU, 배치 크기에 따라)
- **속도 향상**: 10-30배 빠름

### 4. 사용법
```python
# 설치
# pip install vllm

# 단일 GPU
extractor = BatchMAUDEExtractor(
    model_path='Qwen/Qwen2.5-7B-Instruct',
    tensor_parallel_size=1,
    batch_size=32
)

# 다중 GPU (4개 사용)
extractor = BatchMAUDEExtractor(
    model_path='meta-llama/Llama-3.1-70B-Instruct',
    tensor_parallel_size=4,
    batch_size=64
)
```

### 5. 추가 최적화 옵션
- `max_model_len`: 시퀀스 길이 제한 (메모리 절약)
- `gpu_memory_utilization`: GPU 메모리 사용률 조절
- `quantization`: 양자화 (AWQ, GPTQ 등) 지원

### 6. 주의사항
- Chat 템플릿은 모델에 따라 조정 필요 (Qwen, Llama 등)
- GPU 메모리가 부족하면 `batch_size` 또는 `max_model_len` 줄이기
- `tensor_parallel_size`는 사용 가능한 GPU 수와 일치해야 함

# 프롬프팅 관련 문제 (원본)
1. 예외 처리가 없어서 llm 다차원분리에 실패하더라도 그대로 그 행이 빈 채로 넘어감 <- 개선 필요
2. 드는 시간이 너무 많이 걸려서 프롬프트를 좀 크기를 단축시켜야 됨.
    * 실제로는 여기서 더 단축시키기가 힘듦.
3. system prompt는 더 길어져도 한번만 들어가기 때문에 부담 없이 길게 할 수 있음
    * 여기가 주로 만져야 되는 부분(퀄리티 상승을 위해서)

# vLLM 버전에서의 개선사항

## 1. 예외 처리 강화 ✓
- 개별 샘플 실패시에도 다른 샘플은 정상 처리
- `_success`, `_error`, `_raw_response` 필드로 실패 원인 추적
- 실패한 항목 자동 재시도 (최대 2회)

## 2. 처리 시간 대폭 단축 ✓
- **10-30배 빠른 처리 속도**
- 배치 추론으로 GPU 효율 극대화
- 1000개 샘플 기준: Ollama 15-20분 → vLLM 1-2분

## 3. System Prompt 활용
- System prompt에 상세한 가이드라인 추가 가능
- 품질 향상을 위한 예시 및 설명 포함
- 한 번만 인코딩되므로 성능 영향 최소화

## 4. 추가 개선사항
- 실시간 처리 진행률 표시
- 자동 체크포인트 저장
- 상세한 통계 정보 제공
- 메모리 효율적인 처리